In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from matplotlib import pyplot as plt




### Load Autoencoder files

In [ ]:

data_nn = np.loadtxt('Autoencoder_Output_Correct.csv', delimiter=',')

Data_NN_inc = np.loadtxt('Autoencoder_Output_Incorrect.csv', delimiter=',')




### Reshape the data for GMM 

In [ ]:
# % The output dimension of the autoencoder is 4
nDim = 4

# % number of frames in each movement repetition
l = data_nn.shape[1] // nDim

In [ ]:
# Incorrect sequences

n_seq_corr = data_nn.shape[0]  # % number of correct sequences

data = np.tile(np.arange(1, l + 1), n_seq_corr)
data_position = np.empty((nDim, 0))

# % create a row for the time indices of correct data
for i in range(n_seq_corr):
    temp = np.empty((0, nDim))
    for j in range(nDim):
        temp = np.vstack((temp, data_nn[i, j:nDim * l:nDim]))
    data_position = np.hstack((data_position, temp))

data = np.vstack((data, data_position))

In [ ]:

# % Incorrect sequences

n_seq_inc = Data_NN_inc.shape[0]
Data_inc = np.tile(np.arange(1, l + 1), n_seq_inc)
Data_inc_position = np.empty((nDim, 0))

for i in range(n_seq_inc):
    temp = np.empty((0, nDim))
    for j in range(nDim):
        temp = np.vstack((temp, Data_NN_inc[i, j:nDim * l:nDim]))
    Data_inc_position = np.hstack((Data_inc_position, temp))

Data_inc = np.vstack((Data_inc, Data_inc_position))

In [ ]:
# Train GMM
nbStates = 6
nbVar = data.shape[0]
Priors = np.zeros(nbStates)
Mu = np.zeros((nbVar, nbStates))
Sigma = np.zeros((nbVar, nbVar, nbStates))

In [ ]:

# Plot the GMM encoding results
fig, axs = plt.subplots(nbVar - 1, 1, figsize=(7, 7))

for n in range(nbVar - 1):
    axs[n].plot()  # Implement plotGMM1 function
    for j in range(n_seq_corr):
        axs[n].plot(data[n + 1, j * l:(j + 1) * l], color=[0, 0, 0, 0.1], linewidth=0.25)
    axs[n].set_xlim(min(data[0]), max(data[0]))
    axs[n].set_ylim(min(data[n + 1]) - 0.01, max(data[n + 1]) + 0.01)
    axs[n].set_xticks(np.arange(0, 230, 20))
    axs[n].set_yticks(np.arange(-200, 201, 50))
    axs[n].set_xlabel('Time Frame', fontsize=12)
    axs[n].set_ylabel('Angle (Degrees)', fontsize=12)

plt.savefig('../Results/GMM_Encoded_Movements.png', dpi=300)

In [ ]:
# Calculate data loglikelihood
loglikelihood_corr = np.zeros(n_seq_corr)
loglikelihood_inc = np.zeros(n_seq_inc)

for j in range(n_seq_corr):
    # Implement loglik function
    loglikelihood_corr[j] = -loglik(data[:, j * l:(j + 1) * l], nbStates, Priors, Mu, Sigma)

for j in range(n_seq_inc):
    loglikelihood_inc[j] = -loglik(Data_inc[:, j * l:(j + 1) * l], nbStates, Priors, Mu, Sigma)


In [ ]:
# Plot the scaled loglikelihood for correct and incorrect sequences
MAX = max(max(loglikelihood_corr), max(loglikelihood_inc))
MIN = min(min(loglikelihood_corr), min(loglikelihood_inc))
Loglikelihood_sc_corr = 19 * (loglikelihood_corr - MIN) / (MAX - MIN) + 1
Loglikelihood_sc_inc = 19 * (loglikelihood_inc - MIN) / (MAX - MIN) + 1

In [ ]:
plt.figure()
plt.plot(Loglikelihood_sc_corr, 'go', linewidth=2)
plt.plot(Loglikelihood_sc_inc, 'rs', linewidth=2)
plt.xlabel('Sequence Number', fontsize=18)
plt.ylabel('Loglikelihood', fontsize=18)
plt.title('GMM Loglikelihood Scores', fontsize=18)
plt.legend(['Correct Sequences', 'Incorrect Sequences'], fontsize=16, loc='upper left')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().tick_params(axis='both', which='major', labelsize=14)
plt.savefig('../Results/GMM_Loglikelihood_Scores.png', dpi=300)

In [ ]:
# Plot the movement quality scores
mean_abs_corr = np.mean(np.abs(loglikelihood_corr))
std_abs_corr = np.std(np.abs(loglikelihood_corr))

In [ ]:
corr_Y = np.zeros(n_seq_corr)
inc_Y = np.zeros(n_seq_corr)
Correct_Y = np.zeros(n_seq_corr)
Incorrect_Y = np.zeros(n_seq_corr)

In [ ]:

for r in range(n_seq_corr):
    corr_Y[r] = loglikelihood_corr[r] / (mean_abs_corr + 3 * std_abs_corr)
    inc_Y[r] = loglikelihood_inc[r] / (mean_abs_corr + 3 * std_abs_corr)

    Correct_Y[r] = 1 / (1 + np.exp(-3.2 + corr_Y[r]))
    Incorrect_Y[r] = 1 / (1 + np.exp(-3.2 + corr_Y[r]) + (inc_Y[r] - corr_Y[r]) / 10)


In [ ]:
plt.figure()
plt.plot(Correct_Y, 'ko', markerfacecolor='g')
plt.plot(Incorrect_Y, 'ks', markerfacecolor='r')
plt.xlabel('Sequence Number', fontsize=18)
plt.ylabel('Quality Score', fontsize=18)
plt.title('GMM Movement Quality Scores', fontsize=18)
plt.legend(['Correct Sequences', 'Incorrect Sequences'], fontsize=16, loc='lower left')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().tick_params(axis='both', which='major', labelsize=14)
plt.savefig('../Results/GMM_Movement_Quality_Scores.png', dpi=300)

In [ ]:
# Save the labels
np.savetxt('Labels_Correct.csv', Correct_Y, delimiter=',')
np.savetxt('Labels_Incorrect.csv', Incorrect_Y, delimiter=',')